# 103590450 馬茂源 四資四

In [1]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.utils import AnalysisException
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import udf, mean
from pyspark.sql.types import StringType, StructType, StructField, FloatType
import pandas as pd
import numpy as np
import os, math, time
import itertools
import csv
import html
import string

In [2]:
t0 = time.time()

In [3]:
if not os.path.exists('result'):
    os.mkdir('result')
for i in range(1, 5):
    dir_ = 'result/task{}'.format(i)
    if not os.path.exists(dir_):
        os.mkdir(dir_)

In [4]:
conf = (SparkConf()
        #.setMaster('spark://10.100.5.182:7077')
         .setMaster("local[8]")
         .set("spark.executor.memory", "8g")
        .setAppName("hw3"))

In [5]:
try:
    sc = SparkContext(conf=conf)
    sql_sc = SQLContext(sc)
except ValueError:
    pass

## (1) Given the Reuters-21578 dataset, please calculate all kshingles and output the set representation of the text dataset as a matrix. 

In [6]:
file_names = ['./data/reut2-{0:0>3}.sgm'.format(i) for i in range(22)]
file_names = [file_names[0]]

In [7]:
def parser(file):
    news = []
    start = 0
    for i in range(len(file)):
        if file[i:i+6] == '<BODY>':
            start = i+6
        elif file[i:i+7] == '</BODY>':
            news.append(file[start:i])
    return news

In [8]:
news = []
for i in file_names:
    each_news = parser(' '.join(sc.wholeTextFiles(i).collect()[0])) 
    news.extend(each_news)

In [9]:
len(news)

925

In [10]:
news_rdd = sc.parallelize(news)

In [11]:
def tokenizer(text):
    strip_chars = ' –…' + string.punctuation
    result = []
    for token in html.unescape(str(text)).split():
        token = token.lower()
        token = token.strip(strip_chars)
        if not token.isalpha():
            continue
        length = len(token)
        if length < 1 or length > 26:
            continue
        result.append(token)
    return result

In [12]:
def k_shingle(text, k):
    string = ' '.join(tokenizer(text))
    shingles = set([])
    for i in range(len(string)-k + 1):
        shingles.add(string[i:i+k])
    return shingles

In [13]:
shingled_news_rdd = news_rdd.map(lambda x: k_shingle(x, 5))

In [14]:
shingles = list(shingled_news_rdd.reduce(lambda a, b: a.union(b)))
shingles_count = len(shingles)
shingles_count

60359

In [15]:
def encode_shingles(row):
    vector = [0] * shingles_count 
    for shingle, idx in zip(shingles, range(shingles_count)):
        if shingle in row:
            vector[idx] = 1
    return vector

In [ ]:
encoded_news_rdd = shingled_news_rdd.map(encode_shingles)
encoded_news = encoded_news_rdd.toDF(shingles)

In [ ]:
encoded_news.write.csv('task1_result_Transpose.csv', header=True)

## (2) Given the set representation, compute the minhash signatures of all documents using MapReduce.

## (3) Implement the LSH algorithm by MapReduce and output the resulting candidate pairs of similar documents.

## (4) Implement K-nearest neighbor (KNN) search using LSH and compare its performance with linear search.

In [18]:
sc.stop()
print('cost {:.2f} minutes'.format((time.time()-t0)/60))

cost 0.47 minutes
